In [ ]:
# Install dan pastikan openpyxl ada di environment yang tepat
import sys
!{sys.executable} -m pip install openpyxl

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings

warnings.filterwarnings("ignore")

In [17]:
# ----------------------------------------------------------------------
# 2. DEFINISIKAN PARAMETER
# Nama file CSV Anda
FILE_PATH = r"D:\semester-5\Jaringan-syaraf-tiruan\Project\dataset_bbri.xlsx"

# Berapa hari ke belakang yang digunakan untuk memprediksi 1 hari ke depan
LOOK_BACK = 60

# Proporsi pembagian data training (80% training, 20% testing)
TRAIN_SPLIT_RATIO = 0.8

# Parameter untuk Neural Network
EPOCHS = 100
BATCH_SIZE = 16

In [18]:
# ----------------------------------------------------------------------
# 3. FUNGSI HELPER UNTUK MEMBUAT SEQUENCE
def create_dataset(dataset, look_back=1):
    """
    Mengubah array nilai menjadi format dataset sequence.
    Contoh:
    dataset = [1, 2, 3, 4, 5]
    look_back = 2
    Maka hasilnya:
    dataX = [[1, 2], [2, 3], [3, 4]]
    dataY = [3, 4, 5]
    """
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


In [20]:
# ----------------------------------------------------------------------
# 4. MUAT DAN PROSES DATA
print(f"Memuat data dari {FILE_PATH}...")
try:
    df = pd.read_excel(
        FILE_PATH,
        usecols=['Date', 'Close'],
        parse_dates=['Date'],
        index_col='Date'
    )
except FileNotFoundError:
    print(f"Error: File '{FILE_PATH}' tidak ditemukan.")
    print("Pastikan file CSV berada di direktori yang sama dengan skrip ini.")
    exit()

# Menghapus baris dengan data NaN (jika ada)
df = df.dropna()
print("Data berhasil dimuat.")
print(df.head())

# Ambil nilai 'Close' dan ubah menjadi numpy array
dataset = df['Close'].values.reshape(-1, 1)

Memuat data dari D:\semester-5\Jaringan-syaraf-tiruan\Project\dataset_bbri.xlsx...


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [26]:
# 3) Convert to daily frequency & interpolate
s = df["Close"].asfreq('D').interpolate(method='time')
s.head()

Date
2024-11-07    4550.0
2024-11-08    4520.0
2024-11-09    4500.0
2024-11-10    4480.0
2024-11-11    4460.0
Freq: D, Name: Close, dtype: float64

In [27]:
# 4) Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
values = scaler.fit_transform(s.values.reshape(-1,1)).flatten()

In [29]:
# 5) Windowing for multi-step 7-day forecast
lookback = 60
horizon = 7
X, y = [], []
for i in range(len(values) - lookback - horizon + 1):
    X.append(values[i:i+lookback])
    y.append(values[i+lookback:i+lookback+horizon])
X = np.array(X)
y = np.array(y)

In [30]:
# 6) Train-test split
ratio = 0.2
n_test = int(len(X) * ratio)
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]

In [31]:
# 7) Build model LSTM (Sesuai untuk data 3D)
import tensorflow as tf

model = Sequential([
    # Input shape: [samples, time_steps, features] -> [?, 60, 1]
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(50, activation='relu'),
    Dense(horizon) # Output 7 hari ke depan
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
model.summary()

d:\semester-5\Jaringan-syaraf-tiruan\Project\env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input` object so that the model has a defined input shape and can be properly saved summary_ethod(self, input_shape=(1,))


In [ ]:
# 8) Train
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, verbose=1,
callbacks=[tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)])

NameError: name 'model' is not defined

In [ ]:
# Simpan model yang sudah dilatih
model.save("model_forecasting_bbri.keras")
print("Model berhasil disimpan ke file 'model_forecasting_bbri.keras'")

In [ ]:
# 9) Muat model dan lakukan prediksi
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error

# Muat model dari file
loaded_model = load_model("model_forecasting_bbri.keras")
print("Model berhasil dimuat. Melakukan prediksi pada data test...")

# Lakukan prediksi menggunakan model yang sudah dimuat
y_pred = loaded_model.predict(X_test)


y_test_flat = y_test.reshape(-1,1)
y_pred_flat = y_pred.reshape(-1,1)


y_test_inv = scaler.inverse_transform(y_test_flat).flatten()
y_pred_inv = scaler.inverse_transform(y_pred_flat).flatten()


print("MAE:", mean_absolute_error(y_test_inv, y_pred_inv))
print("RMSE:", np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))

Model berhasil dimuat. Melakukan prediksi pada data test...\n

In [ ]:
# Plot model loss history
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Training Loss History")
plt.legend()
plt.show()

NameError: name 'history' is not defined